In [17]:
from model.velocity_and_position.base import *

In [18]:
tr = np.zeros((n_action, n_timestep, n_position, n_velocity, n_velocity))
bins = list(velocity) + [velocity[-1] + (velocity[-1] - velocity[-2])]
after_friction = velocity - friction_factor * velocity  # Shape=(n_velocity,)
for a in action:
    for v_idx, v in enumerate(velocity):
        for t_idx, t in enumerate(timestep):
            for p_idx, p in enumerate(position):
                new_v = after_friction[v_idx] + force[:, a, t_idx, p_idx]  # Shape=(n_sample,)
                new_v = np.clip(new_v, min(velocity), max(velocity))
                tr[a, t_idx, p_idx, v_idx, :], _ = np.histogram(new_v, bins=bins)
tr_ref = normalize_last_column(tr)

In [19]:
print(n_action)

2


In [20]:
print(n_timestep*n_position)

300


In [21]:
tr = np.zeros((n_action, n_timestep, n_position, n_velocity, n_velocity))
bins = list(velocity) + [velocity[-1] + (velocity[-1] - velocity[-2])]
after_friction = velocity - friction_factor * velocity  # Shape=(n_velocity,)
exp_after_friction = np.expand_dims(after_friction[:], tuple(range(len(force.shape))))  # Shape=(1, 1, 1, 1, n_velocity,)
exp_force = np.expand_dims(force, -1)  # Shape=(n_sample, n_action, n_timestep, n_position, 1)
new_v = exp_after_friction + exp_force # Shape=(n_sample, n_action, n_timestep, n_position, n_velocity)
new_v = np.clip(new_v, min_velocity, max_velocity)
for a_idx in range(n_action):
    for t_idx in range(n_timestep):
        for p_idx in range(n_position):
            for v_idx in range(n_velocity):
                idx = a_idx, t_idx, p_idx, v_idx
                tr[a_idx, t_idx, p_idx, v_idx, :], _ = np.histogram(new_v[:, a_idx, t_idx, p_idx, v_idx], bins=bins)
tr = normalize_last_column(tr)

In [22]:
np.expand_dims(after_friction[:], (0, 1, 2, -1)).shape

(1, 1, 1, 20, 1)

In [23]:
np.sum(tr == tr_ref) == len(tr.ravel())

True

In [20]:
n_velocity

20

In [24]:
(np.expand_dims(np.random.random(size=3), (0, 1)) + np.expand_dims(np.random.random(size=(4, 5)), -1)).shape

(4, 5, 3)

In [32]:
np.expand_dims(after_friction[:], tuple(range(len(force.shape)))).shape

(1, 1, 1, 1, 20)

In [41]:
exp_force = np.expand_dims(force[:, :, :, :], -1)

In [37]:
force.shape

(2, 300, 6, 50)

In [39]:
exp_after_friction = np.expand_dims(after_friction[:], tuple(range(len(force.shape))))

In [40]:
exp_after_friction + force

ValueError: operands could not be broadcast together with shapes (1,1,1,1,20) (2,300,6,50) 

In [42]:
exp_force.shape

(2, 300, 6, 50, 1)

In [44]:
(exp_after_friction + exp_force).shape

(2, 300, 6, 50, 20)